# Examples using the ReferenceRangeType & XdIntervalType

**Set the path and import the extended datatype.**

In [37]:
import os
# set the current working directory to the project parent.
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [38]:
from decimal import Decimal
from S3MPython.utils import xsdstub, xmlstub
from S3MPython.xdt import XdQuantityType, XdStringType, XdIntervalType, ReferenceRangeType

The **XdIntervalType** is a generic type that defines an interval (i.e., range) of a comparable nature. An interval is a contiguous subrange of an equivalent base type. Used to determine intervals of dates, times, quantities, etc. Whose datatypes are the same and are ordered. In S3Model, this type is primarily used in defining an interval for reference ranges.

Review the documentation of XdIntervalType.

In [39]:
help(XdIntervalType)

Help on class XdIntervalType in module S3MPython.xdt:

class XdIntervalType(XdAnyType)
 |  XdIntervalType(label: str, invltype: str)
 |  
 |  Generic type defining an interval (i.e. range) of a comparable type.
 |  
 |  An interval is a contiguous subrange of a comparable base type.
 |  Used to define intervals of dates, times, quantities, etc. whose
 |  datatypes are the same and are ordered.
 |  
 |  In S3Model, they are primarily used in defining reference ranges.
 |  The datatype of upper and lower must be set in the model via the invltype
 |  attribute.
 |  
 |  Method resolution order:
 |      XdIntervalType
 |      XdAnyType
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, label: str, invltype: str)
 |      Initialization for Xd* datatypes in S3Model.
 |      
 |      The semantic label (name of the model) is required.
 |  
 |  getModel(self)
 |      Return a XML Schema complexType definition.
 |  
 |  getXMLInstance(self, example=Fa

The **ReferenceRangeType** type defines a named range associated with any ordered data..

Each such range is sensitive to the context, e.g., sex, age, location, and any other factor which affects ranges. May be used to representing high, low, normal, therapeutic, dangerous, critical, etc. ranges that are constrained by an interval represesnted in an **XdIntervalType**

Review the documentation of ReferenceRangeType.

In [40]:
help(ReferenceRangeType)

Help on class ReferenceRangeType in module S3MPython.xdt:

class ReferenceRangeType(XdAnyType)
 |  ReferenceRangeType(label)
 |  
 |  Defines a named range to be associated with any ORDERED datum. Each such range is sensitive to the context,
 |  e.g. sex, age, location, and any other factor which affects ranges.
 |  May be used to represent high, low, normal, therapeutic, dangerous, critical, etc. ranges that are constrained by an interval.
 |  
 |  Method resolution order:
 |      ReferenceRangeType
 |      XdAnyType
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, label)
 |      Initialization for Xd* datatypes in S3Model.
 |      
 |      The semantic label (name of the model) is required.
 |  
 |  getModel(self)
 |      Return a XML Schema complexType definition.
 |  
 |  getXMLInstance(self, example=False)
 |      Return an example XML fragment for this model.
 |  
 |  validate(self)
 |      Every XdType must implement this method.
 | 

[Reference ranges](https://en.wikipedia.org/wiki/Reference_range) can be used with any Ordered type. 

In this tutorial we will use a XdQuantityType model instance to build a model for recording [Total cholesterol](https://en.wikipedia.org/wiki/Reference_ranges_for_blood_tests#Lipids). A real world example would build a Cluster for the entire Lipids table shown from the previous link. We also assume that this is part of a full electronic record and the person's date of birth (age) and sex (male/female) are recorded elsewhere. Normally the semantic definitions will be related directly to accepted clinical guidelines, not Wikipedia, etc. links as we do here for simplicity.

We need a units (*XdStringType*) object to select either; mmol/L or mg/dL. 

We need six reference ranges, one for each units choice available and one for each range; *desireable*, *borderline* and *high*. Ideally there will be reference ranges defined for each age group as well as by gender. That is overkill for our basic tutorial purposes.


In [41]:
# create the units model
u = XdStringType('Concentration Units')
u.docs = "Select the concentration units."
u.definition_url = 'https://chem.libretexts.org/Textbook_Maps/Analytical_Chemistry/Book%3A_Analytical_Chemistry_2.0_(Harvey)/02_Basic_Tools_of_Analytical_Chemistry/2.2%3A_Concentration'
# create the units options
enums = []
enums.append(('mmol/L', 'https://en.wikipedia.org/wiki/Molar_concentration#Units/mmol/L'))
enums.append(('mg/dL', 'https://en.wikipedia.org/wiki/Molar_concentration#Units/mg/dL'))
# note that we need to create URIs for each unit since they often do not exist
u.enums = enums
u.published = True

print(u)

XdStringType : Concentration Units, ID: cky7slk8u002sw4pw4k28133v Published: True


In [42]:
# Create the Quantity model
d = XdQuantityType("Total Cholesterol")
d.docs = "Record the concentration of cholesterol."
d.definition_url = 'https://en.wikipedia.org/wiki/Cholesterol#Cholesterol_testing'
d.units = u
d.min_inclusive = 0  # can't have a negative amount
d.max_inclusive = 500  # this would clearly be out of range
d.fraction_digits = 1 # limit the decimal places
d.total_digits = 5

print(d)

XdQuantityType : Total Cholesterol, ID: cky7slnpu002uw4pw9zhut62m Published: False


In [43]:
# define reference ranges and intervals for the Desireable range
mmol = ReferenceRangeType("Total Cholesterol Desireable Range (mmol/L)")
mmol.definition_url = 'https://www.mayoclinic.org/tests-procedures/cholesterol-test/about/pac-20384601#desireable/mmol'
mmol.is_normal = True # this is considered the normal/desired range

i = XdIntervalType("TCDR (mmol/L)", Decimal)
i.interval_units = ('mmol/L','https://en.wikipedia.org/wiki/Molar_concentration#Units/mmol/L')
i.definition_url = 'https://www.mayoclinic.org/tests-procedures/cholesterol-test/interval'

# be sure to pass upper/lower as a string. It is coerced into the correct type upon assignment
i.lower = '0'
i.upper = '5.2'
i.upper_included = False
i.published = True

mmol.interval = i
mmol.published = True
d.referenceranges = mmol

print(i) # the XdIntervalType model
print(mmol) # the Reference range model

XdIntervalType : TCDR (mmol/L), ID: cky7slray002yw4pwaecec4z5 Published: True
ReferenceRangeType : Total Cholesterol Desireable Range (mmol/L), ID: cky7slrax002ww4pwgyxqsjte Published: True


In [44]:
mg = ReferenceRangeType("Total Cholesterol Desireable Range (mg/dL)")
mg.definition_url = 'https://www.mayoclinic.org/tests-procedures/cholesterol-test/about/pac-20384601#desireable/mg'
mg.is_normal = True # this is considered the normal/desired range
i = XdIntervalType("TCDR (mg/dL)", Decimal)
i.interval_units = ('mg/dL','https://en.wikipedia.org/wiki/Molar_concentration#Units/mg/dL')
i.definition_url = 'https://www.mayoclinic.org/tests-procedures/cholesterol-test/interval'

i.lower = '0'
i.upper = '200'
i.upper_included = False
i.lower_included = False
i.published = True
mg.interval = i
mg.published = True
d.referenceranges = mg

print(i)
print(mg)

XdIntervalType : TCDR (mg/dL), ID: cky7slue70032w4pwnceo4o95 Published: True
ReferenceRangeType : Total Cholesterol Desireable Range (mg/dL), ID: cky7slue70030w4pw6p5vzdcg Published: True


In [45]:
# define two reference ranges for the Borderline high range
mmol = ReferenceRangeType("Total Cholesterol Borderline high Range (mmol/L)")
mmol.definition_url = 'https://www.mayoclinic.org/tests-procedures/cholesterol-test/about/pac-20384601#Borderlinehigh/mmol'
i = XdIntervalType("TCBR (mmol/L)", Decimal)
i.interval_units = ('mmol/L','https://en.wikipedia.org/wiki/Molar_concentration#Units/mmol/L')
i.definition_url = 'https://www.mayoclinic.org/tests-procedures/cholesterol-test/interval'

i.lower = '5.2'
i.upper = '6.2'
i.upper_included = True
i.lower_included = True
i.published = True
mmol.interval = i
mmol.published = True
d.referenceranges = mmol


print(i)
print(mmol)

XdIntervalType : TCBR (mmol/L), ID: cky7sm51f0036w4pw1pf9dk1a Published: True
ReferenceRangeType : Total Cholesterol Borderline high Range (mmol/L), ID: cky7sm51f0034w4pw2w8v18t9 Published: True


In [46]:
mg = ReferenceRangeType("Total Cholesterol Borderline high Range (mg/dL)")
mg.definition_url = 'https://www.mayoclinic.org/tests-procedures/cholesterol-test/about/pac-20384601#Borderlinehigh/mg'
i = XdIntervalType("TCBR (mg/dL)", Decimal)
i.interval_units = ('mg/dL','https://en.wikipedia.org/wiki/Molar_concentration#Units/mg/dL')
i.definition_url = 'https://www.mayoclinic.org/tests-procedures/cholesterol-test/interval'

i.lower = '200'
i.upper = '239'
i.upper_included = True
i.lower_included = True
i.published = True
mg.interval = i
mg.published = True
d.referenceranges = mg

print(i)
print(mg)

XdIntervalType : TCBR (mg/dL), ID: cky7sma4m003aw4pwqaeqak9r Published: True
ReferenceRangeType : Total Cholesterol Borderline high Range (mg/dL), ID: cky7sma4m0038w4pwl3ekmb38 Published: True


In [47]:
# define two reference ranges for the High range
mmol = ReferenceRangeType("Total Cholesterol High Range (mmol/L)")
mmol.definition_url = 'https://www.mayoclinic.org/tests-procedures/cholesterol-test/about/pac-20384601#high/mmol'
i = XdIntervalType("TCHR (mmol/L)", Decimal)
i.interval_units = ('mmol/L','https://en.wikipedia.org/wiki/Molar_concentration#Units/mmol/L')
i.definition_url = 'https://www.mayoclinic.org/tests-procedures/cholesterol-test/interval'

i.lower = '6.2'
i.lower_included = False
i.upper_bounded = False
i.published = True
mmol.interval = i
mmol.published = True
d.referenceranges = mmol


print(i)
print(mmol)

XdIntervalType : TCHR (mmol/L), ID: cky7smh23003ew4pwu7dzy0ta Published: True
ReferenceRangeType : Total Cholesterol High Range (mmol/L), ID: cky7smh22003cw4pwb9jvfno8 Published: True


In [48]:
mg = ReferenceRangeType("Total Cholesterol High Range (mg/dL)")
mg.definition_url = 'https://www.mayoclinic.org/tests-procedures/cholesterol-test/about/pac-20384601#high/mg'
i = XdIntervalType("TCHR (mg/dL)", Decimal)
i.interval_units = ('mg/dL','https://en.wikipedia.org/wiki/Molar_concentration#Units/mg/dL')
i.definition_url = 'https://www.mayoclinic.org/tests-procedures/cholesterol-test/interval'

i.lower = '240'
i.lower_included = False
i.upper_bounded = False
i.published = True
mg.interval = i
mg.published = True
d.referenceranges = mg
d.published = True

print(i)
print(mg)
print(d)

XdIntervalType : TCHR (mg/dL), ID: cky7smlhy003iw4pw4l7xqyl9 Published: True
ReferenceRangeType : Total Cholesterol High Range (mg/dL), ID: cky7smlhx003gw4pw1jqjmdxv Published: True
XdQuantityType : Total Cholesterol, ID: cky7slnpu002uw4pw9zhut62m Published: True


Examine the model. Here we write the model stub (it is not a complete S3Model Data Model) as a file named stub.xsd. 

In [49]:
xsdstub(d)

Writing model for  XdQuantityType : Total Cholesterol, ID: cky7slnpu002uw4pw9zhut62m Published: True  to stub.xsd


When you view the XML Schema model notice that the required namespaces are wrapped using a *root* element. This *root* element is not part of S3Model. It is only used here so that we can demonstrate validation of the generated stub.xml below. 

We need to add data before creating a XML instance.  


In [50]:
# Select a valid units value
d.units.value = 'mg/dL'
# enter a value
d.value = 123.4

xmlstub(d)

Writing data instance for  XdQuantityType : Total Cholesterol, ID: cky7slnpu002uw4pw9zhut62m Published: True  to stub.xml


There are now two files; stub.xsd and stub.xml, in your *examples* directory. You can use any XML validation tool on stub.xml to validate it against stub.xsd.